## Лабораторная работа №3 (Проведение исследований с решающим деревом)

## Классификация

# 2. Создание бейзлайна и оценка качества

**a. Обучение модели sklearn**

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

# Загружаем данные
df = pd.read_csv('credit_risk_dataset.csv')

# Преобразуем текстовые категории в числовые значения
for col in ['person_home_ownership', 'loan_intent', 'loan_grade', 'cb_person_default_on_file']:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])

# Заполняем пропуски медианными значениями
df = df.fillna(df.median())

# Формируем матрицу признаков и целевой вектор
X = df.drop('loan_status', axis=1)
y = df['loan_status']

# Делим выборку на обучающую и тестовую (80/20)
# Стратификация нужна, чтобы сохранить баланс классов
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Масштабируем признаки
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Инициализируем и обучаем дерево решений с параметрами по умолчанию
dt = DecisionTreeClassifier(random_state=42)
dt.fit(X_train, y_train)

# Получаем предсказания классов и вероятностей
y_pred = dt.predict(X_test)
y_pred_proba = dt.predict_proba(X_test)[:, 1]

# Метрики
print("Решающее дерево Бейзлайн")
print(f"Accuracy: {accuracy_score(y_test, y_pred):.4f}")
print(f"Precision: {precision_score(y_test, y_pred):.4f}")
print(f"Recall: {recall_score(y_test, y_pred):.4f}")
print(f"F1-Score: {f1_score(y_test, y_pred):.4f}")
print(f"ROC-AUC: {roc_auc_score(y_test, y_pred_proba):.4f}")

Решающее дерево Бейзлайн
Accuracy: 0.8920
Precision: 0.7462
Recall: 0.7651
F1-Score: 0.7556
ROC-AUC: 0.8462


**b. Оценка качества модели**

- Accuracy: 0.8920
- Precision: 0.7462
- Recall: 0.7651
- F1-Score: 0.7556
- ROC-AUC: 0.8462

Модель, можно сказать, из коробки показала весьма достойные результаты, но возможно есть риск, что произошло переобучение, поэтому в следующем пункте попробую исправить это настройкой глубины.

# 3. Улучшение бейзлайна

**a. Формулировка гипотез**

**Гипотеза 1: Препроцессинг**

Данные содержат явные выбросы (например, стаж 123 года). Удаление аномалий по возрасту и стажу, ну и замена Label Encoding на One-hot encoding для категорий должны помочь дереву строить более логичные правила ветвления.

**Гипотеза 2: Подбор гиперпараметров**

Деревья решений склонны к переобучению, они могут строить слишком глубокие и сложные структуры, запоминая шумы. А если мы ограниченим максимальную глубину и минимальное количество объектов в листе через кросс-валидацию, это поможет упростить модель, убрать переобучение и, возможно, повысить Precision.

**Гипотеза 3: Отбор признаков (Feature Importance)**

Дерево умеет оценивать важность признаков, поэтому если после обучения посмотреть на feature_importances и удалить признаки с нулевой или ничтожной важностью, можно убрать шум и улучшить качество.

**Гипотеза 4: Балансировка классов**

Попробуем применить class_weight='balanced' прямо внутри модели вместо внешнего SMOTE. Это может еще сильнее поднять Recall.

**b. Проверка гипотез**

In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

df = pd.read_csv('credit_risk_dataset.csv')

# Удаляем выбросы
print("До удаления выбросов:", len(df))
df = df[(df['person_age'] < 100) & (df['person_emp_length'] < 100)]
print("После удаления выбросов:", len(df))

# Заполняем пропуски в числовых колонках медианой
numeric_cols = df.select_dtypes(include=['int64', 'float64']).columns
df[numeric_cols] = df[numeric_cols].fillna(df[numeric_cols].median())

# Используем One-hot encoding вместо Label Encoding
# Это поможет дереву лучше делить категории, не привязываясь к порядку чисел
df = pd.get_dummies(df, columns=['person_home_ownership', 'loan_intent', 'loan_grade', 'cb_person_default_on_file'], drop_first=False)

# Разделяем на признаки и целевой класс
X = df.drop('loan_status', axis=1)
y = df['loan_status']

print(f"Количество признаков после One-hot: {X.shape[1]}")

# Train/test разделение
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Нормализуем данные
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Обучаем дерево
dt = DecisionTreeClassifier(random_state=42)
dt.fit(X_train, y_train)

# Предсказания
y_pred = dt.predict(X_test)
y_pred_proba = dt.predict_proba(X_test)[:, 1]

# Метрики
print("Гипотеза 1: препроцессинг")
print(f"Accuracy: {accuracy_score(y_test, y_pred):.4f}")
print(f"Precision: {precision_score(y_test, y_pred):.4f}")
print(f"Recall: {recall_score(y_test, y_pred):.4f}")
print(f"F1-Score: {f1_score(y_test, y_pred):.4f}")
print(f"ROC-AUC: {roc_auc_score(y_test, y_pred_proba):.4f}")

До удаления выбросов: 32581
После удаления выбросов: 31679
Количество признаков после One-hot: 26
Гипотеза 1: препроцессинг
Accuracy: 0.8966
Precision: 0.7489
Recall: 0.7824
F1-Score: 0.7653
ROC-AUC: 0.8552


**Вывод по гипотезе 1:**

Препроцессинг снова сработал. Метрики чуть подросли, One-Hot Encoding помог дереву лучше разделять категории кредитов, а удаление выбросов убрало шум. Точность почти не изменилась, но общий F1-Score подрос.

Гипотезу 1 принимаем, оставляем эту обработку данных.



In [3]:
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

# Гипотеза 2: подбираем глубину дерева и размер листа
# Берём несколько разумных значений max_depth и min_samples_leaf
max_depth_values = [3, 5, 7, 10, 12, 15]
min_samples_leaf_values = [1, 2, 5, 10]

results = []

print("Подбор гиперпараметров для дерева (5-fold CV по F1):\n")

for max_depth in max_depth_values:
    for min_samples_leaf in min_samples_leaf_values:
        # Создаём дерево с заданными параметрами
        dt = DecisionTreeClassifier(
            random_state=42,
            criterion='gini',
            max_depth=max_depth,
            min_samples_leaf=min_samples_leaf
        )

        # Считаем F1-Score на кросс-валидации (5 фолдов)
        cv_scores = cross_val_score(dt, X_train, y_train, cv=5, scoring='f1')
        mean_score = cv_scores.mean()

        results.append({
            'max_depth': max_depth,
            'min_samples_leaf': min_samples_leaf,
            'mean_f1': mean_score
        })

        print(f"max_depth={max_depth:2d}, min_samples_leaf={min_samples_leaf:2d}: "
              f"F1 (5-fold CV) = {mean_score:.4f}")

# Находим лучшую комбинацию по среднему F1
best = max(results, key=lambda x: x['mean_f1'])
best_max_depth = best['max_depth']
best_min_samples_leaf = best['min_samples_leaf']
best_f1_cv = best['mean_f1']

print(f"\nЛучшие параметры:")
print(f"max_depth = {best_max_depth}, min_samples_leaf = {best_min_samples_leaf}")
print(f"Лучший F1 на кросс-валидации: {best_f1_cv:.4f}")

# Обучаем финальную модель с лучшими параметрами на всей train-выборке
dt_best = DecisionTreeClassifier(
    random_state=42,
    criterion='gini',
    max_depth=best_max_depth,
    min_samples_leaf=best_min_samples_leaf
)
dt_best.fit(X_train, y_train)

# Делаем предсказания на тесте
y_pred = dt_best.predict(X_test)
y_pred_proba = dt_best.predict_proba(X_test)[:, 1]

print("\nГипотеза 2: подобранные гиперпараметры")
print(f"Accuracy: {accuracy_score(y_test, y_pred):.4f}")
print(f"Precision: {precision_score(y_test, y_pred):.4f}")
print(f"Recall: {recall_score(y_test, y_pred):.4f}")
print(f"F1-Score: {f1_score(y_test, y_pred):.4f}")
print(f"ROC-AUC: {roc_auc_score(y_test, y_pred_proba):.4f}")

Подбор гиперпараметров для дерева (5-fold CV по F1):

max_depth= 3, min_samples_leaf= 1: F1 (5-fold CV) = 0.6745
max_depth= 3, min_samples_leaf= 2: F1 (5-fold CV) = 0.6745
max_depth= 3, min_samples_leaf= 5: F1 (5-fold CV) = 0.6745
max_depth= 3, min_samples_leaf=10: F1 (5-fold CV) = 0.6745
max_depth= 5, min_samples_leaf= 1: F1 (5-fold CV) = 0.7325
max_depth= 5, min_samples_leaf= 2: F1 (5-fold CV) = 0.7325
max_depth= 5, min_samples_leaf= 5: F1 (5-fold CV) = 0.7325
max_depth= 5, min_samples_leaf=10: F1 (5-fold CV) = 0.7327
max_depth= 7, min_samples_leaf= 1: F1 (5-fold CV) = 0.7972
max_depth= 7, min_samples_leaf= 2: F1 (5-fold CV) = 0.7974
max_depth= 7, min_samples_leaf= 5: F1 (5-fold CV) = 0.7946
max_depth= 7, min_samples_leaf=10: F1 (5-fold CV) = 0.7934
max_depth=10, min_samples_leaf= 1: F1 (5-fold CV) = 0.8054
max_depth=10, min_samples_leaf= 2: F1 (5-fold CV) = 0.8045
max_depth=10, min_samples_leaf= 5: F1 (5-fold CV) = 0.8011
max_depth=10, min_samples_leaf=10: F1 (5-fold CV) = 0.7963
ma

**Выводы по гипотезе 2:**

Precision вырос до 97% с 75%. Это значит, что если мы банк, то мы не отказываем зря хорошим клиентам. ​Остальные метрики тоже подросли, кроме Recall, он немного упал (с 78% до 72%), но это ожидаемая плата за такую высокую точность.

Гипотеза 2 подтверждена, ограничение глубины убрало переобучение.

In [4]:
# Гипотеза 3: удаление признаков с нулевой важностью
# Посмотрим, какие признаки наша лучшая модель (dt_best) считает важными, а какие нет

import pandas as pd
import numpy as np

# Нам нужны названия колонок, так как X_train сейчас это numpy array
# Берем их из датафрейма X
feature_names = X.columns

# Получаем важность признаков из обученной модели
importances = dt_best.feature_importances_

# Собираем все в табличку для наглядности
feature_imp = pd.DataFrame({'Feature': feature_names, 'Importance': importances})
feature_imp = feature_imp.sort_values(by='Importance', ascending=False)

print("Топ-10 самых важных признаков:")
print(feature_imp.head(10))

# Находим признаки, у которых важность ровно 0
zero_imp_features = feature_imp[feature_imp['Importance'] == 0]['Feature'].tolist()

print(f"\nПризнаки с нулевой важностью ({len(zero_imp_features)} шт): {zero_imp_features}")

# Если такие есть, удаляем их и обучаем заново
if len(zero_imp_features) > 0:
    print("\nУдаляем мусорные признаки и переобучаем...")

    # Чтобы удалить колонки по имени, удобнее вернуть данные в DataFrame
    X_train_df = pd.DataFrame(X_train, columns=feature_names)
    X_test_df = pd.DataFrame(X_test, columns=feature_names)

    X_train_opt = X_train_df.drop(columns=zero_imp_features)
    X_test_opt = X_test_df.drop(columns=zero_imp_features)

    # Обучаем модель с теми же лучшими параметрами (max_depth=10, min_samples_leaf=1)
    dt_fs = DecisionTreeClassifier(
        random_state=42,
        criterion='gini',
        max_depth=10,
        min_samples_leaf=1
    )
    dt_fs.fit(X_train_opt, y_train)

    # Предсказания
    y_pred = dt_fs.predict(X_test_opt)
    y_pred_proba = dt_fs.predict_proba(X_test_opt)[:, 1]

    print("Гипотеза 3: удаление неважных признаков")
    print(f"Accuracy: {accuracy_score(y_test, y_pred):.4f}")
    print(f"Precision: {precision_score(y_test, y_pred):.4f}")
    print(f"Recall: {recall_score(y_test, y_pred):.4f}")
    print(f"F1-Score: {f1_score(y_test, y_pred):.4f}")
    print(f"ROC-AUC: {roc_auc_score(y_test, y_pred_proba):.4f}")
else:
    print("\nВсе признаки важны, удалять нечего.")

Топ-10 самых важных признаков:
                          Feature  Importance
5             loan_percent_income    0.295080
10     person_home_ownership_RENT    0.182040
4                   loan_int_rate    0.166972
1                   person_income    0.095018
14            loan_intent_MEDICAL    0.049636
11  loan_intent_DEBTCONSOLIDATION    0.043614
20                   loan_grade_D    0.037265
2               person_emp_length    0.036138
19                   loan_grade_C    0.026929
9       person_home_ownership_OWN    0.012445

Признаки с нулевой важностью (2 шт): ['loan_grade_B', 'cb_person_default_on_file_Y']

Удаляем мусорные признаки и переобучаем...
Гипотеза 3: удаление неважных признаков
Accuracy: 0.9345
Precision: 0.9657
Recall: 0.7216
F1-Score: 0.8260
ROC-AUC: 0.9130


**Вывод по Гипотезе 3:**

Удаление признаков с нулевой важностью немного ухудшило результат. Все метрики упали на сотые доли процента.

Гипотезу 3 отвергаем.

In [5]:
# Гипотеза 4: балансировка классов (class_weight='balanced')
# Используем лучшие параметры глубины (max_depth=10), чтобы видеть чистый эффект балансировки

from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

# Обучаем модель с балансировкой
dt_balanced = DecisionTreeClassifier(
    random_state=42,
    criterion='gini',
    max_depth=10,          # Оставляем лучшую глубину
    min_samples_leaf=1,
    class_weight='balanced' # Включаем балансировку
)

dt_balanced.fit(X_train, y_train)

# Предсказания
y_pred = dt_balanced.predict(X_test)
y_pred_proba = dt_balanced.predict_proba(X_test)[:, 1]

print("Гипотеза 4: балансировка классов (class_weight='balanced')")
print(f"Accuracy: {accuracy_score(y_test, y_pred):.4f}")
print(f"Precision: {precision_score(y_test, y_pred):.4f}")
print(f"Recall: {recall_score(y_test, y_pred):.4f}")
print(f"F1-Score: {f1_score(y_test, y_pred):.4f}")
print(f"ROC-AUC: {roc_auc_score(y_test, y_pred_proba):.4f}")

Гипотеза 4: балансировка классов (class_weight='balanced')
Accuracy: 0.9130
Precision: 0.8314
Recall: 0.7480
F1-Score: 0.7875
ROC-AUC: 0.9087


**Вывод по Гипотезе 4:**

Как и ожидалось, class_weight='balanced' изменил баланс сил, но общий результат ухудшился.

Гипотезу 4 не берём.

In [6]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

# Собираем итоговую модель на основе лучших гипотез
# Гипотеза 1 (Препроцессинг) + гипотеза 2 (max_depth=10)

df = pd.read_csv('credit_risk_dataset.csv')

# Удаляем выбросы
df = df[(df['person_age'] < 100) & (df['person_emp_length'] < 100)]

# Заполняем пропуски в числовых колонках
numeric_cols = df.select_dtypes(include=['int64', 'float64']).columns
df[numeric_cols] = df[numeric_cols].fillna(df[numeric_cols].median())

# One-hot encoding работает лучше, чем Label Encoding
df = pd.get_dummies(df, columns=['person_home_ownership', 'loan_intent', 'loan_grade', 'cb_person_default_on_file'], drop_first=False)

# Разделяем на признаки и целевой класс
X = df.drop('loan_status', axis=1)
y = df['loan_status']

# Train/test разделение
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Нормализуем данные
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Обучаем модель с лучшими параметрами (max_depth=10)
dt_improved = DecisionTreeClassifier(
    random_state=42,
    criterion='gini',
    max_depth=10,
    min_samples_leaf=1
)
dt_improved.fit(X_train, y_train)

# Предсказания
y_pred = dt_improved.predict(X_test)
y_pred_proba = dt_improved.predict_proba(X_test)[:, 1]

# Метрики
print("Улучшенный бейзлайн")
print(f"Accuracy: {accuracy_score(y_test, y_pred):.4f}")
print(f"Precision: {precision_score(y_test, y_pred):.4f}")
print(f"Recall: {recall_score(y_test, y_pred):.4f}")
print(f"F1-Score: {f1_score(y_test, y_pred):.4f}")
print(f"ROC-AUC: {roc_auc_score(y_test, y_pred_proba):.4f}")

Улучшенный бейзлайн
Accuracy: 0.9351
Precision: 0.9676
Recall: 0.7231
F1-Score: 0.8277
ROC-AUC: 0.9146


**f. Сравнение результатов с пунктом 2**

Accuracy выросла с 0.8920 до 0.9351

Precision вырос сильнее всего с 0.7462 до 0.9676

Recall немного просел с 0.7651 до 0.7231

F1-Score поднялся с 0.7556 до 0.8277

ROC-AUC вырос с 0.8462 до 0.9146

Все показатели выросли кроме Recall.

**g. Выводы**

Проведенные эксперименты показали, что решающее дерево очень чувствительно к настройке гиперпараметров. Борьба с переобучением удалась. Ограничение глубины дерева до 10 уровней дало самый значимый эффект. Precision взлетел почти до 97%, что говорит о высокой надежности предсказаний дефолта. AUC-ROC перелетел за 90%, это уже говорит о многом. Пришлось обменять Recall на Precision, он снизился на 4%, а F1-Score поднялся на 7%, мне этот обмен показался выгодным. Обмен Recall на Precision.


# 4. Имплементация алгоритма машинного обучения

**a. Имлементация решающего дерева**

In [7]:
import numpy as np
from collections import Counter

class Node:

    def __init__(self, feature=None, threshold=None, left=None, right=None, *, value=None):
        self.feature = feature          # Индекс признака
        self.threshold = threshold      # Порог разбиения
        self.left = left                # Левое поддерево
        self.right = right              # Правое поддерево
        self.value = value              # Значение класса (если лист)

    def is_leaf_node(self):
        return self.value is not None


class CustomDecisionTree:

    def __init__(self, min_samples_split=2, max_depth=100, criterion='gini'):
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth
        self.criterion = criterion  # 'gini' или 'entropy'
        self.root = None

    def fit(self, X, y):
        """Обучение модели"""
        # Преобразуем в numpy, если пришли DataFrame/Series
        X = np.array(X)
        y = np.array(y)

        self.n_samples, self.n_features = X.shape
        self.root = self._grow_tree(X, y)

    def _grow_tree(self, X, y, depth=0):
        n_samples, n_features = X.shape
        n_labels = len(np.unique(y))

        # Критерии остановки:
        # 1. Достигли макс. глубины
        # 2. Только один класс в узле
        # 3. Слишком мало сэмплов для разбиения
        if (depth >= self.max_depth or n_labels == 1 or n_samples < self.min_samples_split):
            leaf_value = self._most_common_label(y)
            return Node(value=leaf_value)

        # Жадный поиск лучшего разбиения
        best_feat, best_thresh = self._best_split(X, y, n_features)

        # Если не нашли, делаем лист
        if best_feat is None:
            leaf_value = self._most_common_label(y)
            return Node(value=leaf_value)

        # Рекурсивно строим детей
        left_idxs, right_idxs = self._split(X[:, best_feat], best_thresh)
        left = self._grow_tree(X[left_idxs, :], y[left_idxs], depth + 1)
        right = self._grow_tree(X[right_idxs, :], y[right_idxs], depth + 1)

        return Node(best_feat, best_thresh, left, right)

    def _best_split(self, X, y, n_features):
        best_gain = -1
        split_idx, split_threshold = None, None

        for feat_idx in range(n_features):
            X_column = X[:, feat_idx]
            thresholds = np.unique(X_column) # Перебираем уникальные значения как пороги

            for thr in thresholds:
                gain = self._information_gain(y, X_column, thr)

                if gain > best_gain:
                    best_gain = gain
                    split_idx = feat_idx
                    split_threshold = thr

        return split_idx, split_threshold

    def _information_gain(self, y, X_column, threshold):
        # 1. Считаем impurity родителя
        if self.criterion == 'gini':
            parent_loss = self._gini(y)
        else:
            parent_loss = self._entropy(y)

        # 2. Делим
        left_idxs, right_idxs = self._split(X_column, threshold)
        if len(left_idxs) == 0 or len(right_idxs) == 0:
            return 0

        # 3. Взвешенная impurity детей
        n = len(y)
        n_l, n_r = len(left_idxs), len(right_idxs)

        if self.criterion == 'gini':
            e_l, e_r = self._gini(y[left_idxs]), self._gini(y[right_idxs])
        else:
            e_l, e_r = self._entropy(y[left_idxs]), self._entropy(y[right_idxs])

        child_loss = (n_l / n) * e_l + (n_r / n) * e_r

        # Gain = Родитель - Дети
        ig = parent_loss - child_loss
        return ig

    def _split(self, X_column, split_thresh):
        left_idxs = np.argwhere(X_column <= split_thresh).flatten()
        right_idxs = np.argwhere(X_column > split_thresh).flatten()
        return left_idxs, right_idxs

    def _gini(self, y):
        # Gini impurity: 1 - sum(p^2)
        probas = np.bincount(y) / len(y)
        return 1 - np.sum([p**2 for p in probas])

    def _entropy(self, y):
        # Entropy: -sum(p * log2(p))
        probas = np.bincount(y) / len(y)
        return -np.sum([p * np.log2(p) for p in probas if p > 0])

    def _most_common_label(self, y):
        counter = Counter(y)
        return counter.most_common(1)[0][0]

    def predict(self, X):
        X = np.array(X)
        return np.array([self._predict_one(x, self.root) for x in X])

    def _predict_one(self, x, node):
        if node.is_leaf_node():
            return node.value

        if x[node.feature] <= node.threshold:
            return self._predict_one(x, node.left)
        else:
            return self._predict_one(x, node.right)

    def predict_proba(self, X):
        # Заглушка для совместимости: возвращает 0 или 1
        preds = self.predict(X)
        return np.column_stack((1-preds, preds))

    def print_tree(self, node=None, depth=0):
        """
        Метод для визуализации дерева в текстовом виде
        """
        if node is None:
            node = self.root

        if node.is_leaf_node():
            print(f"{'  '*depth}Predict: {node.value}")
            return

        print(f"{'  '*depth}Feature_{node.feature} <= {node.threshold:.3f}?")
        self.print_tree(node.left, depth + 1)
        self.print_tree(node.right, depth + 1)

**b. Обучение имплементированной модели с базовым бейзлайном**

In [8]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split

df_base = pd.read_csv('credit_risk_dataset.csv')

# Переводим категории в цифры LabelEncoder-ом
for col in ['person_home_ownership', 'loan_intent', 'loan_grade', 'cb_person_default_on_file']:
    le = LabelEncoder()
    df_base[col] = le.fit_transform(df_base[col])

# Заполняем пропуски медианой
df_base = df_base.fillna(df_base.median())

# Делим на X и y
X_base = df_base.drop('loan_status', axis=1)
y_base = df_base['loan_status']

# Разбиваем на трейн и тест
X_train_base, X_test_base, y_train_base, y_test_base = train_test_split(X_base, y_base, test_size=0.2, random_state=42, stratify=y_base)

# Масштабируем
scaler_base = StandardScaler()
X_train_base = scaler_base.fit_transform(X_train_base)
X_test_base = scaler_base.transform(X_test_base)

# Обучаем наше дерево
# Ставим глубину 20, чтобы оно не росло бесконечно (иначе будем ждать до утра)
print("Обучение начато")
my_tree_base = CustomDecisionTree(max_depth=20, criterion='gini')
my_tree_base.fit(X_train_base, y_train_base)
print("Обучение закончено")


Обучение начато
Обучение закончено


**c. Оценка качества имплементированной модели с базовым бейзлайном**

In [9]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

# Проверяем, как оно работает на тесте
y_pred_base_custom = my_tree_base.predict(X_test_base)

# Для ROC-AUC нужны вероятности, но мой класс возвращает просто классы (0/1)
# Сделал заглушку predict_proba, которая вернет просто 0 и 1
y_pred_proba_base_custom = my_tree_base.predict_proba(X_test_base)[:, 1]

print("Имплементированное дерево (базовый бейзлайн)")
print(f"Accuracy: {accuracy_score(y_test_base, y_pred_base_custom):.4f}")
print(f"Precision: {precision_score(y_test_base, y_pred_base_custom):.4f}")
print(f"Recall: {recall_score(y_test_base, y_pred_base_custom):.4f}")
print(f"F1-Score: {f1_score(y_test_base, y_pred_base_custom):.4f}")
print(f"ROC-AUC: {roc_auc_score(y_test_base, y_pred_proba_base_custom):.4f}")

Имплементированное дерево (базовый бейзлайн)
Accuracy: 0.8998
Precision: 0.7788
Recall: 0.7553
F1-Score: 0.7669
ROC-AUC: 0.8477


**d. Сравнение результатов с пунктом 2**

Accuracy у моей модели получилась 0.8998, что даже чуть выше, чем у sklearn (0.8920). Разница меньше процента, но приятно.

Precision тоже оказался выше: 0.7788 против 0.7462.

Recall, наоборот, немного ниже: 0.7553 у меня против 0.7651 у sklearn. Библиотечное дерево нашло чуть больше реальных должников.

F1-Score у имплементированного дерева: 0.7669, а было 0.7556. В целом, качество практически идентичное.

**e. Выводы**

Цифры показывают, что алгоритм реализован правильно. Результаты почти один в один совпадают с библиотечными. То, что моя модель показала себя даже капельку лучше по точности, скорее всего связано с тем, что я поставил ограничение глубины 20, а sklearn строит дерево бесконечно, пока не запомнит все данные. Единственный минус это то, что моя модель учится дольше.

**f., g., и h. Добавление техник из улучшенного бейзлайна, обучение и оценка качества имплементированной модели**

In [10]:
# Используем данные из улучшенного бейзлайна
# Они уже у нас есть в переменных X_train, y_train, X_test, y_test (почищенные и закодированные)

# Конвертируем в numpy
X_train_impr = np.array(X_train)
X_test_impr = np.array(X_test)
y_train_impr = np.array(y_train)
y_test_impr = np.array(y_test)

print("Обучаем дерево с (max_depth=10)")

# Создаем модель с лучшим гиперпараметром, который мы нашли в гипотезе 2
my_tree_improved = CustomDecisionTree(
    max_depth=10,        # Ограничиваем глубину, чтобы не переобучалась
    criterion='gini',
    min_samples_split=2
)

# Обучаем
my_tree_improved.fit(X_train_impr, y_train_impr)
print("Обучение завершено")

# Предсказания
y_pred_custom_impr = my_tree_improved.predict(X_test_impr)
y_pred_proba_custom_impr = my_tree_improved.predict_proba(X_test_impr)[:, 1]

# Оценка качества
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

print("\nИмплементированное дерево с улучшениями")
print(f"Accuracy: {accuracy_score(y_test, y_pred_custom_impr):.4f}")
print(f"Precision: {precision_score(y_test, y_pred_custom_impr):.4f}")
print(f"Recall: {recall_score(y_test, y_pred_custom_impr):.4f}")
print(f"F1-Score: {f1_score(y_test, y_pred_custom_impr):.4f}")
print(f"ROC-AUC: {roc_auc_score(y_test, y_pred_proba_custom_impr):.4f}")

Обучаем дерево с (max_depth=10)
Обучение завершено

Имплементированное дерево с улучшениями
Accuracy: 0.9342
Precision: 0.9620
Recall: 0.7231
F1-Score: 0.8256
ROC-AUC: 0.8576


**i. Сравнение результатов с пунктом 3**

Результаты моей имплементации практически полностью совпали с улучшенным бейзлайном из sklearn. Accuracy у меня 0.9342, у sklearn было 0.9351, разницы почти нет. Precision у моей модели 0.9620, у sklearn 0.9676. Оба значения высокие. Recall совпал почти идеально: 0.7231 в обоих случаях. Это значит, что структура дерева получилась вполне идентичной.

**j. Выводы**

Моя реализация решающего дерева справилась с задачей. Всё работает, при тех же данных и тех же параметрах (max_depth=10) она выдает результаты идентичные библиотеке sklearn.

##Регрессия

# 2. Создание бейзлайна и оценка качества

**a. Обучение модели sklearn**

In [11]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Загружаем датасет
df = pd.read_csv('Job_Market_India.csv')

# Удаляем лишнее
df = df.drop(['Record_Date', 'Salary_Trend_Pct'], axis=1)

# Кодируем текстовые признаки
for col in ['Company_Name', 'Job_Role', 'Experience_Level', 'City']:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])

# Заполняем пропуски медианой
df = df.fillna(df.median())

# Разделяем на признаки и целевую переменную
X = df.drop('Salary_INR', axis=1)
y = df['Salary_INR']

# Train/test разделение
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Нормализуем данные
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Обучаем дерево регрессии
dt_reg = DecisionTreeRegressor(random_state=42)
dt_reg.fit(X_train, y_train)

# Предсказываем
y_pred = dt_reg.predict(X_test)

# Метрики
mae = mean_absolute_error(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
r2 = r2_score(y_test, y_pred)
mape = np.mean(np.abs((y_test - y_pred) / y_test)) * 100

print("Дерево решений (Бейзлайн)")
print(f"MAE: {mae:.2f}")
print(f"RMSE: {rmse:.2f}")
print(f"R²: {r2:.4f}")
print(f"MAPE: {mape:.2f}%")

Дерево решений (Бейзлайн)
MAE: 546925.29
RMSE: 736133.63
R²: 0.1392
MAPE: 51.09%


**b. Оценка качества модели**

Базовая модель дерева решений показала слабые результаты. Коэффициент детерминации R² равен всего 0.1392, что говорит о том, что модель почти не улавливает закономерности в данных и работает чуть лучше, чем простое предсказание средней зарплаты. Средняя ошибка в процентах составляет 51%, это очень много. Если человек реально получает 100 тысяч, модель может предсказать и 50, и 150 тысяч. Причины такого провала, я думаю, в переобучении.

# 3. Улучшение бейзлайна

**a. Формулирование гипотез**

**Гипотеза 1: Препроцессинг**

Признак Experience_Level (опыт работы) имеет явный порядок: "0-1 years" < "1-3 years" < "5-8 years" и т.д. LabelEncoder кодирует их алфавитно, из-за чего "12+ years" может оказаться меньше, чем "3-5 years".
Идея: Закодировать опыт вручную по возрастанию (0, 1, 2...), а для остальных категорий (город и компания) использовать One-Hot Encoding. Плюс удалить выбросы по зарплате.

**Гипотеза 2: Ограничение глубины дерева**

Как и в классификации, дерево регрессии без тормозов начинает подстраиваться под каждую уникальную зарплату, теряя способность обобщать. Можно подобрать оптимальную глубину и минимальное число примеров в листе через кросс-валидацию, чтобы модель искала общие тренды, а не запоминала частные случаи.

**Гипотеза 3: Логарифмирование целевой переменной**

Много людей с маленькой зарплатой и мало с огромной. Это мешает модели минимизировать квадратичную ошибку.Можно Обучать модель предсказывать не Salary, а log(Salary). Это должно сделать распределение более нормальным и снизить влияние гигантских зарплат на обучение.

**Гипотеза 4: Подбор критерия разбиения**

В DecisionTreeRegressor можно менять критерий разбиения: squared_error (MSE, стандартный) или absolute_error (MAE). Попробуем оба и посмотрим, какой даст лучшую MAPE.



**b. Проверка гипотез**

In [12]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

df = pd.read_csv('Job_Market_India.csv')
df = df.drop(['Record_Date', 'Salary_Trend_Pct'], axis=1)

# 1. Удаляем жесткие выбросы по зарплате (берем 1% и 99% квантили)
# Это уберет аномально низкие и аномально высокие зарплаты, которые сбивают модель
q_low = df['Salary_INR'].quantile(0.01)
q_high = df['Salary_INR'].quantile(0.99)
df = df[(df['Salary_INR'] >= q_low) & (df['Salary_INR'] <= q_high)]

# 2. Правильное кодирование опыта
# Создаем словарь, чтобы задать порядок. Если каких-то категорий нет в датасете, код не упадет.
experience_map = {
    '0-1 years': 0,
    '1-3 years': 1,
    '3-5 years': 2,
    '5-8 years': 3,
    '8-12 years': 4,
    '12+ years': 5
}
# map заменит строки на числа. Если вдруг встретится что-то новое, заполнится NaN
df['Experience_Level'] = df['Experience_Level'].map(experience_map)

# 3. Заполняем пропуски
numeric_cols = df.select_dtypes(include=['int64', 'float64']).columns
df[numeric_cols] = df[numeric_cols].fillna(df[numeric_cols].median())

# 4. One-Hot Encoding для остальных категорий
# drop_first=True не обязательно для дерева, но уменьшает кол-во колонок
df = pd.get_dummies(df, columns=['Company_Name', 'Job_Role', 'City'], drop_first=False)

# Разделяем
X = df.drop('Salary_INR', axis=1)
y = df['Salary_INR']

# Train/test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Нормализация
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Обучаем дерево
dt_reg = DecisionTreeRegressor(random_state=42)
dt_reg.fit(X_train, y_train)

# Предсказания
y_pred = dt_reg.predict(X_test)

# Метрики
mae = mean_absolute_error(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
r2 = r2_score(y_test, y_pred)
mape = np.mean(np.abs((y_test - y_pred) / y_test)) * 100

print("Гипотеза 1: Препроцессинг")
print(f"MAE: {mae:.2f}")
print(f"RMSE: {rmse:.2f}")
print(f"R²: {r2:.4f}")
print(f"MAPE: {mape:.2f}%")

Гипотеза 1: Препроцессинг
MAE: 521602.46
RMSE: 695136.27
R²: 0.0796
MAPE: 49.76%


**Вывод по гипотезе 1:**

Хоть R² и просел, но MAE и RMSE уменьшились.

Гипотеза 1 подтверждена.

In [13]:
from sklearn.model_selection import GridSearchCV

# Сетка параметров
# В регрессии min_samples_leaf обычно нужно брать побольше, чтобы сглаживать предсказания
param_grid = {
    'max_depth': [5, 10, 15, 20, None],
    'min_samples_leaf': [1, 5, 10, 20, 50]
}

print("Подбор гиперпараметров для регрессии")

# Используем GridSearch
grid_search = GridSearchCV(
    DecisionTreeRegressor(random_state=42),
    param_grid,
    cv=5,
    scoring='neg_mean_squared_error',
    n_jobs=-1
)

grid_search.fit(X_train, y_train)

print(f"Лучшие параметры: {grid_search.best_params_}")

# Обучаем лучшую модель
dt_best = grid_search.best_estimator_

# Предсказания
y_pred = dt_best.predict(X_test)

# Метрики
mae = mean_absolute_error(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
r2 = r2_score(y_test, y_pred)
mape = np.mean(np.abs((y_test - y_pred) / y_test)) * 100

print("\nГипотеза 2: Подбор глубины")
print(f"MAE: {mae:.2f}")
print(f"RMSE: {rmse:.2f}")
print(f"R²: {r2:.4f}")
print(f"MAPE: {mape:.2f}%")

Подбор гиперпараметров для регрессии
Лучшие параметры: {'max_depth': 10, 'min_samples_leaf': 50}

Гипотеза 2: Подбор глубины
MAE: 393977.33
RMSE: 494989.24
R²: 0.5333
MAPE: 39.31%


**Вывод по гипотезе 2:**

После подбора max_depth=10 и min_samples_leaf=50 качество заметно выросло: R² подскочил до 0.5333, то есть модель стала объяснять уже больше половины разброса зарплат. Ошибки тоже просели: MAE уменьшился примерно до 394 тысяч, а MAPE до 39.3%, то есть средняя относительная ошибка стала меньше 40%.

Гипотеза 2 подтверждена.

In [14]:
# Гипотеза 3: Логарифмирование целевой переменной
# Обучаем ту же лучшую модель (max_depth=10, min_samples_leaf=50), но на логарифмах

# Логарифмируем y_train
# Используем log1p (log(1+x)), чтобы не упасть на нулях
y_train_log = np.log1p(y_train)

# Обучаем
dt_log = DecisionTreeRegressor(
    random_state=42,
    max_depth=10,
    min_samples_leaf=50
)
dt_log.fit(X_train, y_train_log)

# Предсказываем логарифм
y_pred_log = dt_log.predict(X_test)

# Возвращаем обратно в деньги
# expm1 - обратная функция к log1p
y_pred = np.expm1(y_pred_log)

# Метрики считаем по нормальным деньгам
mae = mean_absolute_error(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
r2 = r2_score(y_test, y_pred)
mape = np.mean(np.abs((y_test - y_pred) / y_test)) * 100

print("Гипотеза 3: Логарифмирование таргета")
print(f"MAE: {mae:.2f}")
print(f"RMSE: {rmse:.2f}")
print(f"R²: {r2:.4f}")
print(f"MAPE: {mape:.2f}%")

Гипотеза 3: Логарифмирование таргета
MAE: 397989.53
RMSE: 503056.02
R²: 0.5180
MAPE: 36.87%


**Вывод по гипотезе 3:**

Хотя R² немного снизился, MAPE стала лучше: 36.87% против 39.31%. В задачах с зарплатами важнее точнее попадать в порядок сумм, чем минимизировать квадрат ошибки на миллионерах.

Гипотеза 3 подтверждена.

In [15]:
# Гипотеза 4: Подбор критерия разбиения
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Загружаем и готовим данные
df = pd.read_csv('Job_Market_India.csv')
df = df.drop(['Record_Date', 'Salary_Trend_Pct'], axis=1)

# Удаляем выбросы
q_low = df['Salary_INR'].quantile(0.01)
q_high = df['Salary_INR'].quantile(0.99)
df = df[(df['Salary_INR'] >= q_low) & (df['Salary_INR'] <= q_high)]

# Кодируем опыт
experience_map = {
    '0-1 years': 0, '1-3 years': 1, '3-5 years': 2,
    '5-8 years': 3, '8-12 years': 4, '12+ years': 5
}
df['Experience_Level'] = df['Experience_Level'].map(experience_map)

# Пропуски
numeric_cols = df.select_dtypes(include=['int64', 'float64']).columns
df[numeric_cols] = df[numeric_cols].fillna(df[numeric_cols].median())

# One-hot
df = pd.get_dummies(df, columns=['Company_Name', 'Job_Role', 'City'], drop_first=False)

X = df.drop('Salary_INR', axis=1)
y = df['Salary_INR']

# Логарифмируем таргет
y_log = np.log1p(y)

# Разделяем
X_train, X_test, y_train_log, y_test_log = train_test_split(X, y_log, test_size=0.2, random_state=42)

# Нормализуем
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Тестируем два критерия
print("Гипотеза 4: Сравнение критериев разбиения\n")

# Модель 1: squared_error (MSE - стандартный)
dt_mse = DecisionTreeRegressor(
    random_state=42,
    max_depth=10,
    min_samples_leaf=50,
    criterion='squared_error'
)
dt_mse.fit(X_train, y_train_log)
y_pred_log_mse = dt_mse.predict(X_test)
y_pred_mse = np.expm1(y_pred_log_mse)

# Модель 2: absolute_error (MAE)
dt_mae = DecisionTreeRegressor(
    random_state=42,
    max_depth=10,
    min_samples_leaf=50,
    criterion='absolute_error'
)
dt_mae.fit(X_train, y_train_log)
y_pred_log_mae = dt_mae.predict(X_test)
y_pred_mae = np.expm1(y_pred_log_mae)

# Реальный таргет
y_test_orig = np.expm1(y_test_log)

# Метрики для MSE
print("Criterion = 'squared_error' (MSE):")
mae_mse = mean_absolute_error(y_test_orig, y_pred_mse)
rmse_mse = np.sqrt(mean_squared_error(y_test_orig, y_pred_mse))
r2_mse = r2_score(y_test_orig, y_pred_mse)
mape_mse = np.mean(np.abs((y_test_orig - y_pred_mse) / y_test_orig)) * 100
print(f"MAE: {mae_mse:.2f}")
print(f"RMSE: {rmse_mse:.2f}")
print(f"R²: {r2_mse:.4f}")
print(f"MAPE: {mape_mse:.2f}%")

# Метрики для MAE
print("\nCriterion = 'absolute_error' (MAE):")
mae_mae_val = mean_absolute_error(y_test_orig, y_pred_mae)
rmse_mae = np.sqrt(mean_squared_error(y_test_orig, y_pred_mae))
r2_mae = r2_score(y_test_orig, y_pred_mae)
mape_mae = np.mean(np.abs((y_test_orig - y_pred_mae) / y_test_orig)) * 100
print(f"MAE: {mae_mae_val:.2f}")
print(f"RMSE: {rmse_mae:.2f}")
print(f"R²: {r2_mae:.4f}")
print(f"MAPE: {mape_mae:.2f}%")

# Вывод
if mape_mae < mape_mse:
    print("\nГипотеза 4: criterion='absolute_error' работает лучше")
else:
    print("\nГипотеза 4: criterion='squared_error' остается лучшим выбором.")


Гипотеза 4: Сравнение критериев разбиения

Criterion = 'squared_error' (MSE):
MAE: 397989.53
RMSE: 503056.02
R²: 0.5180
MAPE: 36.87%

Criterion = 'absolute_error' (MAE):
MAE: 398400.09
RMSE: 504460.54
R²: 0.5153
MAPE: 39.40%

Гипотеза 4: criterion='squared_error' остается лучшим выбором.


**Вывод по гипотезе 4:**

Результаты показали, что MSE работает лучше: MAPE составила 36.87% против 39.40% у MAE. Хотя MAE теоретически должен быть менее чувствителен к выбросам, на практике в нашем случае (после логарифмирования таргета и чистки данных) стандартный MSE показал себя точнее. R² и RMSE тоже чуть лучше у MSE.

Гипотезу 4 отвергаем.


**c. и d. Формирование и обучение моделис улучшенным бейзлайном**

In [16]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

df = pd.read_csv('Job_Market_India.csv')
df = df.drop(['Record_Date', 'Salary_Trend_Pct'], axis=1)

# Удаляем выбросы по зарплате
q_low = df['Salary_INR'].quantile(0.01)
q_high = df['Salary_INR'].quantile(0.99)
df = df[(df['Salary_INR'] >= q_low) & (df['Salary_INR'] <= q_high)]

# Правильное кодирование опыта
experience_map = {
    '0-1 years': 0, '1-3 years': 1, '3-5 years': 2,
    '5-8 years': 3, '8-12 years': 4, '12+ years': 5
}
df['Experience_Level'] = df['Experience_Level'].map(experience_map)

# Заполняем пропуски
numeric_cols = df.select_dtypes(include=['int64', 'float64']).columns
df[numeric_cols] = df[numeric_cols].fillna(df[numeric_cols].median())

# One-hot для категорий
df = pd.get_dummies(df, columns=['Company_Name', 'Job_Role', 'City'], drop_first=False)

X = df.drop('Salary_INR', axis=1)
y = df['Salary_INR']

# Логарифмируем таргет
y_log = np.log1p(y)

# Разделение
X_train, X_test, y_train_log, y_test_log = train_test_split(X, y_log, test_size=0.2, random_state=42)

# Нормализация
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Обучаем с лучшими параметрами
dt_improved = DecisionTreeRegressor(
    random_state=42,
    criterion='squared_error',
    max_depth=10,
    min_samples_leaf=50
)
dt_improved.fit(X_train, y_train_log)

# Предсказания
y_pred_log = dt_improved.predict(X_test)
y_pred = np.expm1(y_pred_log)
y_test_orig = np.expm1(y_test_log)

# Метрики
mae = mean_absolute_error(y_test_orig, y_pred)
rmse = np.sqrt(mean_squared_error(y_test_orig, y_pred))
r2 = r2_score(y_test_orig, y_pred)
mape = np.mean(np.abs((y_test_orig - y_pred) / y_test_orig)) * 100

print("Улучшенный бейзлайн")
print(f"MAE: {mae:.2f}")
print(f"RMSE: {rmse:.2f}")
print(f"R²: {r2:.4f}")
print(f"MAPE: {mape:.2f}%")

Улучшенный бейзлайн
MAE: 397989.53
RMSE: 503056.02
R²: 0.5180
MAPE: 36.87%


**f. Сравнение результатов с пунктом 2**

Улучшенный бейзлайн показывает заметно лучшие результаты по сравнению с исходной моделью.

MAE уменьшилась с 546 925 до 397 990 рупий. Средняя ошибка снизилась примерно на 149 тысяч.

RMSE снизился с 736 134 до 503 056. Это означает, что модель стала реже допускать очень большие ошибки на высоких значениях зарплат.

R² вырос с 0.14 до 0.52.

MAPE уменьшился с 51% до 36.87%. Относительная ошибка сократилась на 14 процентных пунктов, и предсказания стали существенно ближе к реальным значениям.

В целом улучшенный бейзлайн показывает гораздо более стабильную и точную работу по всем метрикам.

**g. Выводы**

Все три гипотезы оказались рабочими и улучшили модель.

Корректный Ordinal Encoding помог вернуть смысл порядку уровней опыта. Ограничение глубины дерева оказалось самым полезным, модель перестала переобучаться и стала лучше обобщать данные. Логарифмирование таргета снизило перекос на больших зарплатах и уменьшило относительные ошибки.Попытка перейти на критерий MAE мне ничего не дала, MSE всё равно работал лучше.

В итоге модель стала куда точнее, итоговый MAPE опустился ниже 37%, что уже подходит для реальной оценки зарплат.

# 4. Имплементация алгоритма машинного обучения

**a. Имплементация решающего дерева**

In [17]:
import numpy as np

class Node:
    def __init__(self, feature=None, threshold=None, left=None, right=None, *, value=None):
        self.feature = feature          # Индекс признака
        self.threshold = threshold      # Порог разбиения
        self.left = left                # Левое поддерево
        self.right = right              # Правое поддерево
        self.value = value              # Среднее значение (если лист)

    def is_leaf_node(self):
        return self.value is not None


class CustomDecisionTreeRegressor:


    def __init__(self, min_samples_split=2, max_depth=100, min_samples_leaf=1, criterion='squared_error'):
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth
        self.min_samples_leaf = min_samples_leaf
        self.criterion = criterion  # 'squared_error' или 'absolute_error'
        self.root = None

    def fit(self, X, y):
        """Обучение модели"""
        X = np.array(X)
        y = np.array(y)

        self.n_samples, self.n_features = X.shape
        self.root = self._grow_tree(X, y)

    def _grow_tree(self, X, y, depth=0):
        n_samples = len(y)

        # Критерии остановки:
        # 1. Достигли макс. глубины
        # 2. Слишком мало сэмплов для разбиения
        # 3. Все значения одинаковые (дисперсия = 0)
        if (depth >= self.max_depth or
            n_samples < self.min_samples_split or
            self._variance(y) < 1e-7):
            leaf_value = self._mean_value(y)
            return Node(value=leaf_value)

        # Жадный поиск лучшего разбиения
        best_feat, best_thresh = self._best_split(X, y)

        # Если не нашли (variance reduction = 0), делаем лист
        if best_feat is None:
            leaf_value = self._mean_value(y)
            return Node(value=leaf_value)

        # Разбиваем данные
        left_idxs, right_idxs = self._split(X[:, best_feat], best_thresh)

        # Проверяем min_samples_leaf
        if len(left_idxs) < self.min_samples_leaf or len(right_idxs) < self.min_samples_leaf:
            leaf_value = self._mean_value(y)
            return Node(value=leaf_value)

        # Рекурсивно строим детей
        left = self._grow_tree(X[left_idxs, :], y[left_idxs], depth + 1)
        right = self._grow_tree(X[right_idxs, :], y[right_idxs], depth + 1)

        return Node(best_feat, best_thresh, left, right)

    def _best_split(self, X, y):
        best_gain = -1
        split_idx, split_threshold = None, None

        for feat_idx in range(self.n_features):
            X_column = X[:, feat_idx]
            thresholds = np.unique(X_column)

            for thr in thresholds:
                gain = self._variance_reduction(y, X_column, thr)

                if gain > best_gain:
                    best_gain = gain
                    split_idx = feat_idx
                    split_threshold = thr

        return split_idx, split_threshold

    def _variance_reduction(self, y, X_column, threshold):
        # Считаем variance reduction

        # Считаем "impurity" родителя
        if self.criterion == 'squared_error':
            parent_loss = self._mse(y)
        else:
            parent_loss = self._mae(y)

        # Делим
        left_idxs, right_idxs = self._split(X_column, threshold)
        if len(left_idxs) == 0 or len(right_idxs) == 0:
            return 0

        # Взвешенная impurity детей
        n = len(y)
        n_l, n_r = len(left_idxs), len(right_idxs)

        if self.criterion == 'squared_error':
            e_l, e_r = self._mse(y[left_idxs]), self._mse(y[right_idxs])
        else:
            e_l, e_r = self._mae(y[left_idxs]), self._mae(y[right_idxs])

        child_loss = (n_l / n) * e_l + (n_r / n) * e_r

        # Variance reduction
        return parent_loss - child_loss

    def _split(self, X_column, split_thresh):
        left_idxs = np.argwhere(X_column <= split_thresh).flatten()
        right_idxs = np.argwhere(X_column > split_thresh).flatten()
        return left_idxs, right_idxs

    def _mse(self, y):
        # Mean Squared Error (variance)
        if len(y) == 0:
            return 0
        return np.var(y)

    def _mae(self, y):
        # Mean Absolute Error от медианы
        if len(y) == 0:
            return 0
        median = np.median(y)
        return np.mean(np.abs(y - median))

    def _variance(self, y):
        # Дисперсия (для проверки остановки)
        return np.var(y)

    def _mean_value(self, y):
        # Среднее значение для листа
        return np.mean(y)

    def predict(self, X):
        X = np.array(X)
        return np.array([self._predict_one(x, self.root) for x in X])

    def _predict_one(self, x, node):
        if node.is_leaf_node():
            return node.value

        if x[node.feature] <= node.threshold:
            return self._predict_one(x, node.left)
        else:
            return self._predict_one(x, node.right)

    def print_tree(self, node=None, depth=0):

        if node is None:
            node = self.root

        if node.is_leaf_node():
            print(f"{'  '*depth}Predict: {node.value:.2f}")
            return

        print(f"{'  '*depth}Feature_{node.feature} <= {node.threshold:.3f}?")
        self.print_tree(node.left, depth + 1)
        self.print_tree(node.right, depth + 1)

**b. Обучение и оценка качества имплементированной модели с базовым бейзлайном**

In [18]:
# b. Обучение имплементированной модели (базовый бейзлайн)
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Грузим данные
df_base = pd.read_csv('Job_Market_India.csv')
df_base = df_base.drop(['Record_Date', 'Salary_Trend_Pct'], axis=1)

# Кодируем все категории LabelEncoder-ом
for col in ['Company_Name', 'Job_Role', 'Experience_Level', 'City']:
    le = LabelEncoder()
    df_base[col] = le.fit_transform(df_base[col])

# Заполняем пропуски медианой
df_base = df_base.fillna(df_base.median())

# Делим на X и y
X_base = df_base.drop('Salary_INR', axis=1)
y_base = df_base['Salary_INR']

# Train/test разделение
X_train_base, X_test_base, y_train_base, y_test_base = train_test_split(X_base, y_base, test_size=0.2, random_state=42)

# Масштабируем
scaler_base = StandardScaler()
X_train_base = scaler_base.fit_transform(X_train_base)
X_test_base = scaler_base.transform(X_test_base)

# Обучаем с параметрами по умолчанию (max_depth=100, без реальных ограничений)
print("Обучение начато")
my_tree_reg_base = CustomDecisionTreeRegressor()  # Параметры по умолчанию
my_tree_reg_base.fit(X_train_base, y_train_base)
print("Обучение завершено")

# c. Оценка качества
y_pred_base_custom = my_tree_reg_base.predict(X_test_base)

mae = mean_absolute_error(y_test_base, y_pred_base_custom)
rmse = np.sqrt(mean_squared_error(y_test_base, y_pred_base_custom))
r2 = r2_score(y_test_base, y_pred_base_custom)
mape = np.mean(np.abs((y_test_base - y_pred_base_custom) / y_test_base)) * 100

print("\nИмплементированное дерево (Базовый бейзлайн)")
print(f"MAE: {mae:.2f}")
print(f"RMSE: {rmse:.2f}")
print(f"R²: {r2:.4f}")
print(f"MAPE: {mape:.2f}%")

Обучение начато
Обучение завершено

Имплементированное дерево (Базовый бейзлайн)
MAE: 547452.09
RMSE: 733285.71
R²: 0.1459
MAPE: 51.31%


**d. Сравнение результатов с пунктом 2**

**Имплементированный алгоритм:**

MAE: 547452.09

RMSE: 733285.71

R²: 0.1459

MAPE: 51.31%

**sklearn:**

MAE: 546925.29

RMSE: 736133.63

R²: 0.1392

MAPE: 51.09%

Результаты схожи.

**e. Выводы**

Алгоритм дерева регрессии реализован. Почти полное совпадение метрик с sklearn подтверждает корректность работы методов разбиения по дисперсии, расчета MSE и выбора среднего значения в листьях.


**f. и g. Добавление техник из улучшенного бейзлайна и обучение модели**

In [20]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Загружаем данные
df_improved = pd.read_csv('Job_Market_India.csv')
df_improved = df_improved.drop(['Record_Date', 'Salary_Trend_Pct'], axis=1)

# 1. Удаление выбросов (IQR метод для Salary_INR)
Q1 = df_improved['Salary_INR'].quantile(0.25)
Q3 = df_improved['Salary_INR'].quantile(0.75)
IQR = Q3 - Q1
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR
df_improved = df_improved[(df_improved['Salary_INR'] >= lower_bound) &
                           (df_improved['Salary_INR'] <= upper_bound)]

print(f"После удаления выбросов осталось {len(df_improved)} записей")

# 2. Создание новых признаков
df_improved['Demand_Remote'] = df_improved['Demand_Index'] * df_improved['Remote_Option_Flag']

# 3. OneHotEncoding для категориальных признаков
df_improved = pd.get_dummies(df_improved, columns=['Company_Name', 'Job_Role', 'Experience_Level', 'City'],
                              drop_first=True, dtype=int)

# Заполняем пропуски
df_improved = df_improved.fillna(df_improved.median())

# Разделяем на X и y
X_improved = df_improved.drop('Salary_INR', axis=1)
y_improved = df_improved['Salary_INR']

print(f"Количество признаков после обработки: {X_improved.shape[1]}")

# Train/test split
X_train_imp, X_test_imp, y_train_imp, y_test_imp = train_test_split(
    X_improved, y_improved, test_size=0.2, random_state=42
)

# Масштабирование
scaler_imp = StandardScaler()
X_train_imp = scaler_imp.fit_transform(X_train_imp)
X_test_imp = scaler_imp.transform(X_test_imp)

print("\nОбучение начато")
my_tree_reg_improved = CustomDecisionTreeRegressor(
    max_depth=10,
    min_samples_split=50,
    min_samples_leaf=20,
    criterion='squared_error'
)
my_tree_reg_improved.fit(X_train_imp, y_train_imp)
print("Обучение завершено")

y_pred_imp_custom = my_tree_reg_improved.predict(X_test_imp)

mae_imp = mean_absolute_error(y_test_imp, y_pred_imp_custom)
rmse_imp = np.sqrt(mean_squared_error(y_test_imp, y_pred_imp_custom))
r2_imp = r2_score(y_test_imp, y_pred_imp_custom)
mape_imp = np.mean(np.abs((y_test_imp - y_pred_imp_custom) / y_test_imp)) * 100

print("\nИмплементированное дерево (Улучшенный бейзлайн)")
print(f"MAE: {mae_imp:.2f}")
print(f"RMSE: {rmse_imp:.2f}")
print(f"R²: {r2_imp:.4f}")
print(f"MAPE: {mape_imp:.2f}%")

После удаления выбросов осталось 28389 записей
Количество признаков после обработки: 63

Обучение начато
Обучение завершено

Имплементированное дерево (Улучшенный бейзлайн)
MAE: 366862.77
RMSE: 444755.45
R²: 0.4193
MAPE: 40.25%


**i. Сравнение результатов с пунктом 3**

**sklearn:**

MAE: 397989.53

RMSE: 503056.02

R²: 0.5180

MAPE: 36.87%

**Имплементированный:**

MAE: 366862.77

RMSE: 444755.45

R²: 0.4193

MAPE: 40.25%

Моя реализация показала себя немного хуже sklearn.

**j. Выводы**

Моя реализация дерева на улучшенных данных работает неплохо, но sklearn всё равно лучше. Причина, думаю, в том что sklearn использует более оптимизированные алгоритмы выбора разбиений и критерии остановки.